In [1]:
import os
import cv2

import numpy as np

In [2]:
def cv_show(img):
    # 图像读取
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 全局配置

In [9]:
image = cv2.imread('data/answerSheet/test.png')

ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

## 图像预处理

In [10]:
contours_img = image.copy()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)

cv_show(image)
cv_show(blurred)
cv_show(edged)

## 轮廓检测

In [11]:
cnts = cv2.findContours(edged.copy(),
                        cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE)[1]

cv2.drawContours(contours_img, cnts, -1, (0, 255, 255), 3)
cv_show(contours_img)

In [12]:
docCnt = None

# 确保检测到了，并根据轮廓大小排序，做近似并为透视变换做准备
if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02*peri, True)
        
        if len(approx) == 4:
            docCnt = approx
            break

## 透视变换

In [14]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")

    # 按顺序找到对应坐标0123分别是 左上，右上，右下，左下
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # 计算右上和左下
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

In [15]:
def four_point_transform(image, pts):
    # 获取输入坐标点
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    # 计算输入的w和h值
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # 变换后对应坐标位置
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    
    # 计算变换矩阵
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    
    return warped

In [17]:
warped = four_point_transform(gray, docCnt.reshape(4, 2))
cv_show(warped)

## 自适应阈值处理

In [21]:
thresh = cv2.threshold(warped, 0, 255,
                       cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

cv_show(thresh)

## 再次轮廓检测

In [27]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
        
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes), 
                                        key=lambda b: b[1][i], 
                                        reverse=reverse))
    return cnts, boundingBoxes

In [28]:
thresh_contours = warped.copy()

# 找到每一个圆圈轮廓
cnts = cv2.findContours(thresh.copy(),
                        cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE)[1]

cv2.drawContours(thresh_contours, cnts, -1, (0, 255, 255), 3)
cv_show(thresh_contours)

In [29]:
questionCnts = []

for c in cnts:
    (x, y ,w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

# 对轮廓从上到下进行排序
questionCnts = sort_contours(questionCnts, method="top-to-bottom")[0]

## 检测答案并打分

In [ ]:
# 每排5个选项
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    cnts = sort_contours(questionCnts[i: i+5])[0]
    bubbled = None
    
    # 遍历每一个结果
    for (j, c) in enumerate(cnts):
        # 使用mask来判断结果
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        cv_show(mask)